# CIFAR-10 classifier from nnBuilder

This is a simple implementation of a CIFAR-10 classifier using the nnBuilder framework. Trying to reproduce the network of https://www.tensorflow.org/tutorials/deep_cnn/ (Work in progress)

In [1]:
import tensorflow as tf
import numpy as np
import sys, time
sys.path.append('../nnBuilder')
from nnLayer import *
from nnInput import *
from nnTrainer import *
from nnHandler import *
from _nnUtils import *

In [2]:
data=Layer(x=None,type="CIFAR_10")
preprocess_layers=[]
preprocess_layers.append(dict(type="Random_Crop",shape=[1,24,24,3]))
preprocess_layers.append(dict(type="Reshape",shape=[24,24,3]))
preprocess_layers.append(dict(type="Random_Horizontal_Flip"))
preprocess_layers.append(dict(type="Random_Brightness"))
preprocess_layers.append(dict(type="Random_Contrast"))
preprocess_layers.append(dict(type="Whitening"))
preprocess_layers_batch=[layer.copy() for layer in preprocess_layers if layer["type"]!="Reshape"]
for layer in preprocess_layers_batch:
    layer["type"]="Batch_"+layer["type"]
preprocess_layers_batch[0]["shape"]=preprocess_layers_batch[0]["shape"][1:]
#No preprocessing (40ms/batch, 95% gpu, 25% cpu)
batch_def_0=dict(x=data,type="Batch_Slice",batch=128)
#No input pipeline (200ms/batch, 30% gpu, 35% cpu)
batch_def_1=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch)
#With input pipeline and custom batch maker (memory problem if more than 1 thread)
#1 Thread: (200ms/batch, 30% gpu, 35% cpu)
#2 Threads: (180ms/batch, ~40% gpu, 40% cpu)
#3 Threads: (175ms/batch, ~45% gpu, 45% cpu)
#4+ Threads: OOM
#Stopped Threads: (13ms/batch, ~90% gpu, 20% cpu)
batch_def_2=dict(x=data,type="Batch_Slice",batch=128,out_features=preprocess_layers_batch
                +[dict(type="Pipeline",num_threads=2)])
#With input pipeline and custom batch maker (quick memory fix, not working)
batch_def_3=dict(x=data,type="Batch_Slice",batch=128,in_features=[dict(type="Pipeline",num_threads=1)],
                 out_features=preprocess_layers_batch+[dict(type="Pipeline",num_threads=1)])
#With input pipeline and tensorflow'batch maker (labels broken)
batch_def_4=dict(x=data,type="Pipeline",batch=128,make_batch=True,shuffle=True,in_features=preprocess_layers)

batch=Layer(**batch_def_2)
layers=[]
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Convolution",pad="SAME",window=5,stride=1,size=64,relu=True,out_features=[
              dict(type="Pool",pad="SAME",window=3,stride=2,pool_type="max"),
              dict(type="Local_Response_Norm")]))
layers.append(dict(type="Relu",size=384))
layers.append(dict(type="Relu",size=192))
layers.append(dict(type="Linear",size=10,in_features=["Dropout"]))
network_def=dict(type="Network",layers=layers)
network=Layer(x=batch,**network_def)
trainer=ClassifierTrainer(network=network,optimizer="adam",finish=True)
sess=SessManager(data,batch,network,trainer)#,input_network)
sess.start()

Starting new session


InternalError: Dst tensor is not initialized.
	 [[Node: zeros = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [50000,32,32,3] values: [[[0 0 0]]]...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'zeros', defined at:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_instance
    app.start()
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Program Files\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 162, in start
    super(ZMQIOLoop, self).start()
  File "C:\Program

In [ ]:
trainer.train(0,keep_rate=1.)
trainer.eval_error()
for i in range(5):
    t0=time.time()
    trainer.train(100,keep_rate=1.)
    trainer.train(0,keep_rate=1.)
    trainer.eval_error() #Training error, reaches about 10%
    print("Time: %s"%(time.time()-t0))

In [ ]:
network.save() #The full network definition
save=network.save() #The full network definition
assert(Layer(x=batch,**save).save()==save) #Consistency
print(save==network_def) #Saving adds stuff, should still be equivalent
save